# FileSystem (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Runtime.fs

In [ ]:
open Common

## Operators

In [ ]:
let inline (</>) a b = System.IO.Path.Combine (a, b)

## createTempDirectoryName

In [ ]:
let inline createTempDirectoryName () =
    let root =
        match System.Reflection.Assembly.GetEntryAssembly().GetName().Name with
        | assemblyName -> assemblyName

    System.IO.Path.GetTempPath ()
    </> $"!{root}"
    </> string (newGuidFromDateTime System.DateTime.Now)

In [ ]:
//// test

createTempDirectoryName ()
|> _contains System.IO.Path.DirectorySeparatorChar

C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3560-6055-65ecbdaa23d1


## createTempDirectory

In [ ]:
let inline createTempDirectory () =
    let tempFolder = createTempDirectoryName ()
    let result = System.IO.Directory.CreateDirectory tempFolder

    let getLocals () =
        $"tempFolder: {tempFolder} / result: {({|
            Exists = result.Exists
            CreationTime = result.CreationTime
        |})} {getLocals ()}"

    trace Debug (fun () -> "createTempDirectory") getLocals

    tempFolder

In [ ]:
//// test

let tempDirectory = createTempDirectory ()

Directory.Exists tempDirectory
|> _equal true

03:13:35 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3576-7657-7c32f608173a / result: { CreationTime = 2023-07-30 3:13:35 AM
  Exists = true }
True


## waitForFileAccess

In [ ]:
let inline waitForFileAccess access path =
    let fileAccess, fileShare =
        access
        |> Option.defaultValue (System.IO.FileAccess.ReadWrite, System.IO.FileShare.Read)

    let rec loop retry = async {
        try
            use _ = new System.IO.FileStream (
                path,
                System.IO.FileMode.Open,
                fileAccess,
                fileShare
            )
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "waitForFileAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let inline lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use stream = new System.IO.FileStream (
        path,
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
    stream.Seek (0L, SeekOrigin.Begin) |> ignore
    trace Debug (fun () -> "_4") getLocals
    stream.WriteByte 49uy
    trace Debug (fun () -> "_5") getLocals
    stream.Flush ()
    trace Debug (fun () -> "_6") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = path |> waitForFileAccess None
    trace Debug (fun () -> "5") getLocals
    let! text = File.ReadAllTextAsync path |> Async.AwaitTask
    trace Debug (fun () -> "6") getLocals
    do! child
    trace Debug (fun () -> "7") getLocals
    return retries, text
}
|> Async.runWithTimeout 3000
|> function
    | Some (retries, text) ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 200)
        
        text |> _equal "1"
        
        true
    | _ -> false
|> _equal true

03:13:36 #2 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3627-2776-291cd3e7ff1d / result: { CreationTime = 2023-07-30 3:13:36 AM
  Exists = true }
03:13:36 #3 [Debug] 1
03:13:36 #4 [Debug] 2
03:13:36 #5 [Debug] _1
03:13:36 #6 [Debug] 3
03:13:36 #7 [Debug] _2
03:13:36 #8 [Debug] 4
03:13:36 #9 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3627-2776-291cd3e7ff1d\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3627-2776-291cd3e7ff1d\test.txt' because it is being used by another process.
03:13:37 #10 [Warn] waitForFileAccess / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3627-2776-291cd3e7ff1d\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3627-2776-291cd3e7ff1d\test.txt' because it is being u

## deleteDirectoryAsync

In [ ]:
let inline deleteDirectoryAsync path =
    let rec loop retry = async {
        try
            System.IO.Directory.Delete (path, true)
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "deleteDirectoryAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test"

let inline lockDirectory () = async {
    trace Debug (fun () -> "_1") getLocals
    System.IO.File.WriteAllText (path </> "test.txt", "0")
    use _ = new System.IO.FileStream (
        path </> "test.txt",
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    Directory.CreateDirectory path |> ignore
    trace Debug (fun () -> "2") getLocals
    let! child = lockDirectory () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = deleteDirectoryAsync path
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries
}
|> Async.runWithTimeout 3000
|> function
    | Some retries ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 0)
            (if Runtime.isWindows () then 150 else 0)

        true
    | _ -> false
|> _equal true

03:13:38 #18 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3886-8640-842b768ac749 / result: { CreationTime = 2023-07-30 3:13:38 AM
  Exists = true }
03:13:38 #19 [Debug] 1
03:13:38 #20 [Debug] 2
03:13:38 #21 [Debug] 3
03:13:38 #22 [Debug] _1
03:13:38 #23 [Debug] _2
03:13:38 #24 [Debug] 4
03:13:38 #25 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3886-8640-842b768ac749\test / ex: System.IO.IOException: The process cannot access the file 'test.txt' because it is being used by another process.
03:13:40 #26 [Warn] deleteDirectoryAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-3886-8640-842b768ac749\test / ex: System.IO.IOException: The process cannot access the file 'test.txt' because it is being used by another process.
03:13:40 #27 [Debug] _3
03:13:40 #28 [Debug] 5
03:13:40 #29 [Debug] 6
127
127
127
True


## deleteFileAsync

In [ ]:
let inline deleteFileAsync path =
    let rec loop retry = async {
        try
            System.IO.File.Delete path
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () = $"path: {path} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "deleteFileAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"

let inline lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (
        path,
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries = deleteFileAsync path
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries
}
|> Async.runWithTimeout 3000
|> function
    | Some retries ->
        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 0)
            (if Runtime.isWindows () then 150 else 0)

        true
    | _ -> false
|> _equal true

03:13:41 #30 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4126-2679-2fa9cc0b4b9e / result: { CreationTime = 2023-07-30 3:13:41 AM
  Exists = true }
03:13:41 #31 [Debug] 1
03:13:41 #32 [Debug] 2
03:13:41 #33 [Debug] 3
03:13:41 #34 [Debug] _1
03:13:41 #35 [Debug] _2
03:13:41 #36 [Debug] 4
03:13:41 #37 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4126-2679-2fa9cc0b4b9e\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4126-2679-2fa9cc0b4b9e\test.txt' because it is being used by another process.
03:13:42 #38 [Warn] deleteFileAsync / path: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4126-2679-2fa9cc0b4b9e\test.txt / ex: System.IO.IOException: The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4126-2679-2fa9cc0b4b9e\test.txt' because it is bei

## moveFileAsync

In [ ]:
let inline moveFileAsync newPath oldPath =
    let rec loop retry = async {
        try
            System.IO.File.Move (oldPath, newPath)
            return retry
        with ex ->
            if retry % 100 = 0 then
                let getLocals () =
                    $"oldPath: {oldPath} / newPath: {newPath} / ex: {ex |> printException} / {getLocals ()}"
                trace Warn (fun () -> "moveFileAsync") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let tempFolder = createTempDirectory ()
let path = tempFolder </> "test.txt"
let newPath = tempFolder </> "test2.txt"

let inline lockFile () = async {
    trace Debug (fun () -> "_1") getLocals
    use _ = new System.IO.FileStream (
        path,
        System.IO.FileMode.Open,
        System.IO.FileAccess.ReadWrite,
        System.IO.FileShare.None
    )
    trace Debug (fun () -> "_2") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_3") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    do! File.WriteAllTextAsync (path, "0") |> Async.AwaitTask
    trace Debug (fun () -> "2") getLocals
    let! child = lockFile () |> Async.StartChild
    trace Debug (fun () -> "3") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "4") getLocals
    let! retries1 = path |> moveFileAsync newPath
    trace Debug (fun () -> "5") getLocals
    let! retries2 = newPath |> waitForFileAccess None
    trace Debug (fun () -> "6") getLocals
    let! text = File.ReadAllTextAsync newPath |> Async.AwaitTask
    trace Debug (fun () -> "7") getLocals
    do! child
    trace Debug (fun () -> "8") getLocals
    return retries1, retries2, text
}
|> Async.runWithTimeout 3000
|> function
    | Some (retries1, retries2, text) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 100 else 0)
            (if Runtime.isWindows () then 150 else 0)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 0 else 100)
            (if Runtime.isWindows () then 0 else 200)
        
        text |> _equal "0"
        
        true
    | _ -> false
|> _equal true

03:13:43 #42 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4369-6956-6d7fa19711a9 / result: { CreationTime = 2023-07-30 3:13:43 AM
  Exists = true }
03:13:43 #43 [Debug] 1
03:13:43 #44 [Debug] 2
03:13:43 #45 [Debug] 3
03:13:43 #46 [Debug] _1
03:13:43 #47 [Debug] _2
03:13:43 #48 [Debug] 4
03:13:43 #49 [Warn] moveFileAsync / oldPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4369-6956-6d7fa19711a9\test.txt / newPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4369-6956-6d7fa19711a9\test2.txt / ex: System.IO.IOException: The process cannot access the file because it is being used by another process.
03:13:45 #50 [Warn] moveFileAsync / oldPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4369-6956-6d7fa19711a9\test.txt / newPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-4369-6956-6d7fa19711a9\test2.txt / ex: System.IO.IOException: The process cannot access the 

## watchDirectory

In [ ]:
[<RequireQualifiedAccess>]
type FileSystemChangeType =
    | Error
    | Changed
    | Created
    | Deleted
    | Renamed

[<RequireQualifiedAccess>]
type FileSystemChange =
    | Error of exn: exn
    | Changed of path: string * content: string option
    | Created of path: string * content: string option
    | Deleted of path: string
    | Renamed of oldPath: string * (string * string option)


let inline watchDirectoryWithFilter filter shouldReadContent path =
    let fullPath = System.IO.Path.GetFullPath path
    let getLocals () = $"fullPath: {fullPath} / filter: {filter} / {getLocals ()}"

    let watcher =
        new System.IO.FileSystemWatcher (
            Path = fullPath,
            NotifyFilter = filter,
            EnableRaisingEvents = true,
            IncludeSubdirectories = true
        )

    let inline getEventPath (path : string) =
        path |> String.trim |> String.replace fullPath "" |> String.trimStart [| '/'; '\\' |]

    let inline ticks () =
        System.DateTime.UtcNow.Ticks

    let changedStream =
        AsyncSeq.subscribeEvent
            watcher.Changed
            (fun event ->
                ticks (),
                [ FileSystemChange.Changed (getEventPath event.FullPath, None) ]
            )

    let deletedStream =
        AsyncSeq.subscribeEvent
            watcher.Deleted
            (fun event ->
                ticks (),
                [ FileSystemChange.Deleted (getEventPath event.FullPath) ]
            )

    let createdStream =
        AsyncSeq.subscribeEvent
            watcher.Created
            (fun event ->
                let path = getEventPath event.FullPath
                ticks (), [
                    FileSystemChange.Created (path, None)
                    if Runtime.isWindows () then
                        FileSystemChange.Changed (path, None)
                ])

    let renamedStream =
        AsyncSeq.subscribeEvent
            watcher.Renamed
            (fun event ->
                ticks (), [
                    FileSystemChange.Renamed (
                        getEventPath event.OldFullPath,
                        (getEventPath event.FullPath, None)
                    )
                ]
            )

    let errorStream =
        AsyncSeq.subscribeEvent
            watcher.Error
            (fun event -> ticks (), [ FileSystemChange.Error (event.GetException ()) ])

    let inline readContent fullPath =
        let rec loop retry = async {
            try
                if retry > 0
                then do!
                    fullPath
                    |> waitForFileAccess (Some (
                        System.IO.FileAccess.Read,
                        System.IO.FileShare.Read
                    ))
                    |> Async.runWithTimeoutAsync 10000
                    |> Async.Ignore
                return! System.IO.File.ReadAllTextAsync fullPath |> Async.AwaitTask |> Async.map Some
            with ex ->
                let getLocals () = $"retry: {retry} / ex: {ex |> printException} / {getLocals ()}"
                trace Error (fun () -> $"watchWithFilter / readContent") getLocals
                if retry = 0
                then return! loop (retry + 1)
                else return None
        }
        loop 0

    let stream =
        [
            changedStream
            deletedStream
            createdStream
            renamedStream
            errorStream
        ]
        |> FSharp.Control.AsyncSeq.mergeAll
        |> FSharp.Control.AsyncSeq.map (fun (t, events) ->
            events
            |> List.fold
                (fun (i, events) event ->
                    i + 1L,
                    (t + i, event) :: events)
                (0L, [])
            |> snd
            |> List.rev
        )
        |> FSharp.Control.AsyncSeq.concatSeq
        |> FSharp.Control.AsyncSeq.mapAsyncParallel (fun (t, event) -> async {
            match shouldReadContent, event with
            | true, FileSystemChange.Changed (path, _) ->
                let! content = fullPath </> path |> readContent
                return t, FileSystemChange.Changed (path, content)
            | true, FileSystemChange.Created (path, _) ->
                let! content = fullPath </> path |> readContent
                return t, FileSystemChange.Created (path, content)
            | true, FileSystemChange.Renamed (oldPath, (newPath, _)) ->
                let! content = fullPath </> newPath |> readContent
                return t, FileSystemChange.Renamed (oldPath, (newPath, content))
            | _ -> return t, event
        })

    let disposable =
        newDisposable (fun () ->
            trace Debug (fun () -> "watchWithFilter / Disposing watch stream") getLocals
            watcher.EnableRaisingEvents <- false
            watcher.Dispose ()
        )

    stream, disposable

let inline watchDirectory path =
    watchDirectoryWithFilter
        (System.IO.NotifyFilters.FileName
        // ||| System.IO.NotifyFilters.DirectoryName
        // ||| System.IO.NotifyFilters.Attributes
        //// ||| System.IO.NotifyFilters.Size
        ||| System.IO.NotifyFilters.LastWrite
        //// ||| System.IO.NotifyFilters.LastAccess
        // ||| System.IO.NotifyFilters.CreationTime
        // ||| System.IO.NotifyFilters.Security
        )
        path

### testEventsRaw (test)

In [ ]:
//// test

let inline testEventsRaw
    (watchFn : bool -> string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable)
    write
    =
    let tempDirectory = createTempDirectory ()
    let stream, disposable = watchFn true tempDirectory

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let inline iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 250
        return! write tempDirectory
    }

    try
        run
        |> Async.runWithTimeout 60000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync tempDirectory |> Async.Ignore |> Async.RunSynchronously

    let eventsLog =
        events
        |> Seq.toList
        |> List.sortBy fst
        |> List.fold
            (fun (prev, acc) (ticks, event) ->
                ticks, (ticks, (if prev = 0L then 0L else ticks - prev), event) :: acc
            )
            (0L, [])
        |> snd
        |> List.rev
        |> List.map (fun (diff, n, event) ->
            let text = $"{n} / {diff} / {event}"
            if text |> String.length <= 100
            then text
            else text |> String.substring 0 100 |> String.replace "\n" ""
        )
        |> String.concat "\n"
    let getLocals () = $"eventsLog: \n{eventsLog} / {getLocals ()}"
    trace Debug (fun () -> "testEventsRaw") getLocals

    events
    |> Seq.toList
    |> List.sortBy fst
    |> List.map snd
    |> List.fold
        (fun acc event ->
            match acc, event with
            | FileSystemChange.Changed (lastPath, Some lastContent) as lastEvent :: acc,
                FileSystemChange.Changed (path, Some content)
                when lastPath = path && content |> String.startsWith lastContent
                ->
                event :: acc
            | _ -> event :: acc
        )
        []
    |> List.rev

#### fast (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"a{i}") |> Async.AwaitTask

    do! Async.Sleep 250

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"b{i}") |> Async.AwaitTask

    do! Async.Sleep 250

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 250

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"c{i}") |> Async.AwaitTask

    do! Async.Sleep 250

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 250
}

let inline run () =
    let events = testEventsRaw watchDirectory write

    events
    |> _sequenceEqual [
        FileSystemChange.Created ("file1.txt", Some "a1")
        FileSystemChange.Changed ("file1.txt", Some "a1")
        FileSystemChange.Created ("file2.txt", Some "a2")
        FileSystemChange.Changed ("file2.txt", Some "a2")

        FileSystemChange.Changed ("file1.txt", Some "b1")
        FileSystemChange.Changed ("file2.txt", Some "b2")

        FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "b1"))
        FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "b2"))

        FileSystemChange.Changed ("file_1.txt", Some "c1")
        FileSystemChange.Changed ("file_2.txt", Some "c2")

        FileSystemChange.Deleted "file_1.txt"
        FileSystemChange.Deleted "file_2.txt"
    ]

run
|> retryFn 3
|> _equal (Some ())

03:13:52 #56 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-5224-2488-269a22039a14 / result: { CreationTime = 2023-07-30 3:13:52 AM
  Exists = true }
03:13:53 #57 [Error] watchWithFilter / readContent / retry: 0 / ex: System.AggregateException: One or more errors occurred. (The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-5224-2488-269a22039a14\file_1.txt' because it is being used by another process.) / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-5224-2488-269a22039a14 / filter: FileName, LastWrite
FSharpOption<Unit>
      Value: <null>
03:13:54 #58 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0313-5224-2488-269a22039a14 / filter: FileName, LastWrite
03:13:54 #59 [Debug] testEventsRaw / eventsLog: 
0 / 638262944325836129 / Created ("file1.txt", Some "a1")
1 / 638262944325836130 / Change

#### slow (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    let contents =
        [1..n]
        |> List.map (string >> String.replicate 1_000_000)

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{contents.[i - 1]}a") |> Async.AwaitTask

    do! Async.Sleep 1500

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{contents.[i - 1]}b") |> Async.AwaitTask

    do! Async.Sleep 1500

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 1500

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{contents.[i - 1]}c") |> Async.AwaitTask

    do! Async.Sleep 1500

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    do! Async.Sleep 1500
}

let inline run () =
    let events =
        testEventsRaw watchDirectory write
        |> List.map (function
            | FileSystemChange.Changed (path, Some content) ->
                FileSystemChange.Changed (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            | FileSystemChange.Created (path, Some content) ->
                FileSystemChange.Created (path, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
            | FileSystemChange.Renamed (oldPath, (newPath, Some content)) ->
                FileSystemChange.Renamed (
                    oldPath,
                    (newPath, content |> Seq.distinct |> Seq.map string |> String.concat "" |> Some)
                )
            | event -> event
        )

    events
    |> _sequenceEqual [
        FileSystemChange.Created ("file1.txt", Some "1a")
        FileSystemChange.Changed ("file1.txt", Some "1a")
        FileSystemChange.Created ("file2.txt", Some "2a")
        FileSystemChange.Changed ("file2.txt", Some "2a")

        FileSystemChange.Changed ("file1.txt", Some "1b")
        FileSystemChange.Changed ("file2.txt", Some "2b")

        FileSystemChange.Renamed ("file1.txt", ("file_1.txt", Some "1b"))
        FileSystemChange.Renamed ("file2.txt", ("file_2.txt", Some "2b"))

        FileSystemChange.Changed ("file_1.txt", Some "1c")
        FileSystemChange.Changed ("file_2.txt", Some "2c")

        FileSystemChange.Deleted "file_1.txt"
        FileSystemChange.Deleted "file_2.txt"
    ]

run
|> retryFn 3
|> _equal (Some ())

03:14:02 #60 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-0200-0047-05fb34d080ad / result: { CreationTime = 2023-07-30 3:14:02 AM
  Exists = true }
03:14:02 #61 [Error] watchWithFilter / readContent / retry: 0 / ex: System.AggregateException: One or more errors occurred. (The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-0200-0047-05fb34d080ad\file1.txt' because it is being used by another process.) / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-0200-0047-05fb34d080ad / filter: FileName, LastWrite
03:14:02 #62 [Error] watchWithFilter / readContent / retry: 0 / ex: System.AggregateException: One or more errors occurred. (The process cannot access the file 'C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-0200-0047-05fb34d080ad\file1.txt' because it is being used by another process.) / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230

### testEventsSorted (test)

In [ ]:
//// test

let inline sortEvent event =
    match event with
    | FileSystemChange.Error _ -> 0
    | FileSystemChange.Created _ -> 1
    | FileSystemChange.Changed _ -> 2
    | FileSystemChange.Renamed (_oldPath, _) -> 3
    | FileSystemChange.Deleted _ -> 4

let inline formatEvents events =
    events
    |> Seq.toList
    |> List.sortBy (snd >> sortEvent)
    |> List.choose (fun (ticks, event) ->
        match event with
        | FileSystemChange.Error _ ->
            None
        | FileSystemChange.Changed (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Changed)
        | FileSystemChange.Created (path, _) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Created)
        | FileSystemChange.Deleted path ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Deleted)
        | FileSystemChange.Renamed (_oldPath, (path, _)) ->
            Some (ticks, System.IO.Path.GetFileName path, nameof FileSystemChangeType.Renamed)
    )
    |> List.sortBy (fun (_, path, _) -> path)
    |> List.distinctBy (fun (_, path, event) -> path, event)

let inline testEventsSorted
    (watchFn : string -> FSharp.Control.AsyncSeq<int64 * FileSystemChange> * IDisposable)
    write
    =
    let path = createTempDirectory ()
    let stream, disposable = watchFn path

    let events = System.Collections.Concurrent.ConcurrentBag ()

    let inline iter () =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async { events.Add event })

    let run = async {
        let! _ = iter () |> Async.StartChild
        do! Async.Sleep 250
        return! write path
    }

    try
        run
        |> Async.runWithTimeout 5000
        |> _equal (Some ())
    finally
        disposable.Dispose ()
        deleteDirectoryAsync path |> Async.Ignore |> Async.RunSynchronously

    let events = formatEvents events

    let eventMap =
        events
        |> List.map (fun (ticks, path, event) -> path, (event, ticks))
        |> List.groupBy fst
        |> List.map (fun (path, events) ->
            let event, _ticks =
                events
                |> List.map snd
                |> List.sortByDescending snd
                |> List.head

            path, event
        )
        |> Map.ofList

    let eventList =
        events
        |> List.map (fun (_ticks, path, event) -> path, event)

    eventMap, eventList

#### create and delete (test)

In [ ]:
//// test

let inline write path = async {
    let n = 3

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory false) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file1.txt", nameof FileSystemChangeType.Deleted

        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Deleted

        "file3.txt", nameof FileSystemChangeType.Created
        "file3.txt", nameof FileSystemChangeType.Changed
        "file3.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Deleted
        "file2.txt", nameof FileSystemChangeType.Deleted
        "file3.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

03:14:13 #77 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-1309-0987-004dfdc2434b / result: { CreationTime = 2023-07-30 3:14:13 AM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
03:14:13 #78 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-1309-0987-004dfdc2434b / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
( file3.txt, Created )
( file3.txt, Changed )
( file3.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
      - Key: file3.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>


#### change (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory false) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file1.txt", nameof FileSystemChangeType.Deleted

        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Deleted
        "file2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

03:14:15 #79 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-1589-8963-8451fd1d35be / result: { CreationTime = 2023-07-30 3:14:15 AM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
03:14:16 #80 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-1589-8963-8451fd1d35be / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file1.txt, Deleted )
( file2.txt, Created )
( file2.txt, Changed )
( file2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Deleted
      - Key: file2.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>


#### rename (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory false) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed

        "file_1.txt", nameof FileSystemChangeType.Renamed
        "file_1.txt", nameof FileSystemChangeType.Deleted

        "file_2.txt", nameof FileSystemChangeType.Renamed
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Changed
        "file_1.txt", nameof FileSystemChangeType.Deleted
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

03:14:19 #81 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-1937-3720-3619e8846406 / result: { CreationTime = 2023-07-30 3:14:19 AM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
03:14:19 #82 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-1937-3720-3619e8846406 / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>


#### full (test)

In [ ]:
//// test

let inline write path = async {
    let n = 2

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file{i}.txt", "") |> Async.AwaitTask

    for i = 1 to n do
        do! path </> $"file{i}.txt" |> moveFileAsync (path </> $"file_{i}.txt") |> Async.Ignore

    for i = 1 to n do
        do! System.IO.File.WriteAllTextAsync (path </> $"file_{i}.txt", $"{i}") |> Async.AwaitTask

    for i = 1 to n do
        do! deleteFileAsync (path </> $"file_{i}.txt") |> Async.Ignore
    
    do! Async.Sleep 150
}

let inline run () =
    let eventMap, eventList = testEventsSorted (watchDirectory false) write

    [
        "file1.txt", nameof FileSystemChangeType.Created
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Created
        "file2.txt", nameof FileSystemChangeType.Changed

        "file_1.txt", nameof FileSystemChangeType.Changed
        "file_1.txt", nameof FileSystemChangeType.Renamed
        "file_1.txt", nameof FileSystemChangeType.Deleted

        "file_2.txt", nameof FileSystemChangeType.Changed
        "file_2.txt", nameof FileSystemChangeType.Renamed
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> _sequenceEqual eventList

    [
        "file1.txt", nameof FileSystemChangeType.Changed
        "file2.txt", nameof FileSystemChangeType.Changed
        "file_1.txt", nameof FileSystemChangeType.Deleted
        "file_2.txt", nameof FileSystemChangeType.Deleted
    ]
    |> Map.ofList
    |> _sequenceEqual eventMap

run
|> retryFn 3
|> _equal (Some ())

03:14:23 #83 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-2353-5319-5797d208b557 / result: { CreationTime = 2023-07-30 3:14:23 AM
  Exists = true }
FSharpOption<Unit>
      Value: <null>
03:14:24 #84 [Debug] watchWithFilter / Disposing watch stream / fullPath: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230730-0314-2353-5319-5797d208b557 / filter: FileName, LastWrite
FSharpList<Tuple<String,String>>
( file1.txt, Created )
( file1.txt, Changed )
( file2.txt, Created )
( file2.txt, Changed )
( file_1.txt, Changed )
( file_1.txt, Renamed )
( file_1.txt, Deleted )
( file_2.txt, Changed )
( file_2.txt, Renamed )
( file_2.txt, Deleted )
FSharpMap<String,String>
      - Key: file1.txt
        Value: Changed
      - Key: file2.txt
        Value: Changed
      - Key: file_1.txt
        Value: Deleted
      - Key: file_2.txt
        Value: Deleted
FSharpOption<Unit>
      Value: <null>
